In [1]:
# -*- coding: utf-8
# The Intercept Brasil (https://theintercept.com/brasil/)
# Reinaldo Chaves (@paidatocandeira)
# Acessa a base de empresas e sócios da Receita e filtra apenas as relacionadas com segurança
#

In [1]:
import pandas as pd
import unidecode

Função para retirar acentuação 

In [2]:
def f(str):
    return (unidecode.unidecode(str))

# Carrega empresas e socios

Da base criada com informações da Receita Federal. Extração da base aqui (https://github.com/reichaves/curso_knight_python_2017/blob/master/roboempresa.ipynb)

In [5]:
socios_brasil = pd.read_csv("socios_br_4_1.csv",sep=',',encoding = 'utf-8', converters={'cnpj': lambda x: str(x)})

In [6]:
empresas_brasil = pd.read_csv("empresas_br_4_1.csv",sep=',',encoding = 'utf-8', converters={'cnpj': lambda x: str(x)})

# Carrega empresas com segurança e outros itens no nome

In [6]:
mask = empresas_brasil['nome_empresa'].str.contains("SEGURANCA", na=False)

In [7]:
segura = empresas_brasil[mask]

In [8]:
segura.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33024 entries, 1548 to 8952135
Data columns (total 4 columns):
Unnamed: 0      33024 non-null int64
cnpj            33024 non-null object
nome_empresa    33024 non-null object
estado          33024 non-null object
dtypes: int64(1), object(3)
memory usage: 1.3+ MB


In [9]:
mask = empresas_brasil['nome_empresa'].str.contains("VIGILANCIA", na=False)

In [10]:
vigi = empresas_brasil[mask]

In [11]:
vigi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4850 entries, 1790 to 8951645
Data columns (total 4 columns):
Unnamed: 0      4850 non-null int64
cnpj            4850 non-null object
nome_empresa    4850 non-null object
estado          4850 non-null object
dtypes: int64(1), object(3)
memory usage: 189.5+ KB


In [12]:
mask = empresas_brasil['nome_empresa'].str.contains("TRANSPORTE DE VALORES", na=False)

In [13]:
transp = empresas_brasil[mask]

In [14]:
transp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95 entries, 84114 to 8591149
Data columns (total 4 columns):
Unnamed: 0      95 non-null int64
cnpj            95 non-null object
nome_empresa    95 non-null object
estado          95 non-null object
dtypes: int64(1), object(3)
memory usage: 3.7+ KB


Reune dataframes

In [16]:
empresas_seguranca1 = pd.merge(segura, vigi, left_on='cnpj', right_on='cnpj', how='outer')

In [17]:
empresas_seguranca = pd.merge(empresas_seguranca1, transp, left_on='cnpj', right_on='cnpj', how='outer')

In [18]:
empresas_seguranca.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35581 entries, 0 to 35580
Data columns (total 10 columns):
Unnamed: 0_x      33024 non-null float64
cnpj              35581 non-null object
nome_empresa_x    33024 non-null object
estado_x          33024 non-null object
Unnamed: 0_y      4850 non-null float64
nome_empresa_y    4850 non-null object
estado_y          4850 non-null object
Unnamed: 0        95 non-null float64
nome_empresa      95 non-null object
estado            95 non-null object
dtypes: float64(3), object(7)
memory usage: 3.0+ MB


In [19]:
empresas_seguranca.reset_index().head()

,index,Unnamed: 0_x,cnpj,nome_empresa_x,estado_x,Unnamed: 0_y,nome_empresa_y,estado_y,Unnamed: 0,nome_empresa,estado
0,0,1548.0,00529528000152,SEGURA MEDICINA E SEGURANCA DO TRABALHO LTDA - ME,Acre,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1671.0,02197190000104,SUATS SEGURANCA - EIRELI,Acre,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1720.0,05562482000197,ASSOCIACAO DOS ENGENHEIROS DA INDUSTRIA E DE S...,Acre,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1732.0,05564936000169,AMAZONIA SEGURANCA LTDA - ME,Acre,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1752.0,03912598000183,QUALIVIDA ASSESSORIA E CONSULTORIA EM SEGURANC...,Acre,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
empresas_seguranca.to_csv('empresas_seguranca_ajustar.csv', index=False)

Limpeza no arquivo retira colunas que não serão usadas neste levantamento

In [3]:
empresas_seguranca_final = pd.read_csv("empresas_seguranca_ajustado.csv",sep=';',encoding = 'utf-8', converters={'cnpj': lambda x: str(x)})

In [4]:
empresas_seguranca_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35581 entries, 0 to 35580
Data columns (total 3 columns):
cnpj            35581 non-null object
nome_empresa    35581 non-null object
estado          35581 non-null object
dtypes: object(3)
memory usage: 834.0+ KB


Faz série de exclusões de termos que não têm relação com segurança pública

In [8]:
extracao1 = empresas_seguranca_final[empresas_seguranca_final['nome_empresa'].str.contains("SEGURANCA DO TRABALHO") == False]
extracao2 = extracao1[extracao1['nome_empresa'].str.contains("SEGURANCA VEICULAR") == False]
extracao3 = extracao2[extracao2['nome_empresa'].str.contains("SEGURANCA OCUPACIONAL") == False]
extracao4 = extracao3[extracao3['nome_empresa'].str.contains("COOPERATIVA") == False]
extracao5 = extracao4[extracao4['nome_empresa'].str.contains("SEGURANCA E INSTRUMENTACAO") == False]
extracao6 = extracao5[extracao5['nome_empresa'].str.contains("PRODUTOS DE SEGURANCA") == False]
extracao7 = extracao6[extracao6['nome_empresa'].str.contains("EQUIPAMENTOS DE SEGURANCA") == False]
extracao8 = extracao7[extracao7['nome_empresa'].str.contains("MATERIAIS DE SEGURANCA") == False]
extracao9 = extracao8[extracao8['nome_empresa'].str.contains("SEGURANCA E MANUTENCAO") == False]
extracao10 = extracao9[extracao9['nome_empresa'].str.contains("SEGURANCA E MEDICINA") == False]
extracao11 = extracao10[extracao10['nome_empresa'].str.contains("SAUDE OCUPACIONAL") == False]
extracao12 = extracao11[extracao11['nome_empresa'].str.contains("MEDICINA DO TRABALHO") == False]
extracao13 = extracao12[extracao12['nome_empresa'].str.contains("SAUDE") == False]
extracao14 = extracao13[extracao13['nome_empresa'].str.contains("MEDICINA") == False]
extracao15 = extracao14[extracao14['nome_empresa'].str.contains("SEGURANCA AUTOMOTIVA") == False] 
extracao16 = extracao15[extracao15['nome_empresa'].str.contains("MEIO AMBIENTE") == False] 
extracao17 = extracao16[extracao16['nome_empresa'].str.contains("VIDROS") == False] 
extracao18 = extracao17[extracao17['nome_empresa'].str.contains("INCENDIO") == False] 
extracao19 = extracao18[extracao18['nome_empresa'].str.contains("MATERIAL DE SEGURANCA") == False] 
extracao20 = extracao19[extracao19['nome_empresa'].str.contains("SEGURANCA DA INFORMACAO") == False] 
extracao21 = extracao20[extracao20['nome_empresa'].str.contains("RESERVATORIOS") == False] 
extracao22 = extracao21[extracao21['nome_empresa'].str.contains("AUTOMACAO") == False] 
extracao23 = extracao22[extracao22['nome_empresa'].str.contains("CALCADOS DE SEGURANCA") == False] 
extracao24 = extracao23[extracao23['nome_empresa'].str.contains("VEICULAR") == False] 
extracao25 = extracao24[extracao24['nome_empresa'].str.contains("EDITORA") == False] 
extracao26 = extracao25[extracao25['nome_empresa'].str.contains("INFORMATICA") == False] 
extracao27 = extracao26[extracao26['nome_empresa'].str.contains("GESTAO AMBIENTAL") == False] 
extracao28 = extracao27[extracao27['nome_empresa'].str.contains("SINALIZACAO") == False] 
extracao29 = extracao28[extracao28['nome_empresa'].str.contains("RADIOPROTECAO") == False] 
extracao30 = extracao29[extracao29['nome_empresa'].str.contains("BIO SEGURANCA") == False] 
extracao31 = extracao30[extracao30['nome_empresa'].str.contains("BIOSEGURANCA") == False]
extracao32 = extracao31[extracao31['nome_empresa'].str.contains("PERICIAS") == False]
extracao33 = extracao32[extracao32['nome_empresa'].str.contains("ELETRON") == False]
extracao34 = extracao33[extracao33['nome_empresa'].str.contains("SISTEMA") == False]

Salva como CSV o filtro feito

In [17]:
extracao34.to_csv('empresas_seguranca.csv', index=False)

Adiciona os sócios dessas empresas

In [7]:
empres_segur = pd.read_csv("empresas_seguranca.csv",sep=',',encoding = 'utf-8', converters={'cnpj': lambda x: str(x)})

In [8]:
segura_e_socios = pd.merge(empres_segur, socios_brasil, left_on='cnpj', right_on='cnpj')

In [9]:
segura_e_socios.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33097 entries, 0 to 33096
Data columns (total 9 columns):
cnpj            33097 non-null object
nome_empresa    33097 non-null object
estado_x        33097 non-null object
Unnamed: 0      33097 non-null int64
indicador       33097 non-null int64
documento       33097 non-null object
qualificacao    33097 non-null int64
socio           33097 non-null object
estado_y        33097 non-null object
dtypes: int64(3), object(6)
memory usage: 2.5+ MB


In [10]:
segura_e_socios.to_csv('empresas_seguranca_br.csv', index=False)

Seleciona apenas do RJ

In [12]:
segura_e_socios_rj = segura_e_socios[(segura_e_socios['estado_x'] == 'Rio de Janeiro')]

In [13]:
segura_e_socios_rj.to_csv('resultados/empresas_seguranca_rj.csv', index=False)